In [1]:
import pandas as pd

# Práca
bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\df_bridges_2024_all.csv"

# Load the CSV data into a DataFrame
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')

In [2]:
import plotly.graph_objs as go

def extract_elevation_data(gpx_file):
    elevations = []
    distances = []
    with open(gpx_file, "r", encoding="utf-8") as file:
        content = file.read()
        # Split into trkpt segments
        trkpt_segments = content.split("<trkpt")
        previous_lat, previous_lon = None, None
        total_distance = 0.0
        for segment in trkpt_segments[1:]:
            # Extract latitude and longitude
            lat_start = segment.find('lat="') + 5
            lat_end = segment.find('"', lat_start)
            lon_start = segment.find('lon="') + 5
            lon_end = segment.find('"', lon_start)
            ele_start = segment.find("<ele>") + 5
            ele_end = segment.find("</ele>")

            if lat_start > 4 and lon_start > 4 and ele_start > 4:
                latitude = float(segment[lat_start:lat_end])
                longitude = float(segment[lon_start:lon_end])
                elevation = float(segment[ele_start:ele_end])

                if previous_lat is not None and previous_lon is not None:
                    # Calculate distance between points (simple approximation)
                    from math import radians, sin, cos, sqrt, atan2
                    R = 6371000  # Earth radius in meters
                    lat1, lon1 = radians(previous_lat), radians(previous_lon)
                    lat2, lon2 = radians(latitude), radians(longitude)
                    dlat = lat2 - lat1
                    dlon = lon2 - lon1
                    a = sin(dlat / 2)**2 + cos(lat1) * \
                        cos(lat2) * sin(dlon / 2)**2
                    c = 2 * atan2(sqrt(a), sqrt(1 - a))
                    distance = R * c
                    total_distance += distance

                elevations.append(elevation)
                # Convert to kilometers
                distances.append(total_distance / 1000)

                previous_lat, previous_lon = latitude, longitude

    return distances, elevations

In [3]:
# Assuming df_bridges_all is your DataFrame
columns_to_convert = ['Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Normálna_zaťažiteľnosť',
                      'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t', 'Počet_otvorov',
                      'Dĺžka_nosnej_konštrukcie_m', 'Dĺžka_premostenia', 'Volná_šírka_mosta',
                      'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2', 'Plocha_mosta_m2']

# Convert specified columns to float
df_bridges_all[columns_to_convert] = df_bridges_all[columns_to_convert].apply(
    pd.to_numeric, errors='coerce')

# Convert Rok_postavenia to integer, assign NaN if not possible
df_bridges_all['Rok_postavenia'] = pd.to_numeric(
    df_bridges_all['Rok_postavenia'], errors='coerce')

In [4]:
df_bridges_all["Správca_úseku"].value_counts().keys()

Index(['SSC - IVSC ZA', 'SSC - IVSC KE', 'SSC - IVSC BB', 'BBRSC-JUH',
       'BBRSC-STRED', 'BBRSC-SEVER', 'SSC - IVSC BA', 'SC TN SK - TN',
       'SUC PSK - HE', 'SUC PSK - PO', 'SC TN SK - PD', 'SUC TTSK - TT',
       'SUC PSK - PP', 'SUC PSK - BJ', 'SC TN SK - PB', 'SC ŽSK - ORA',
       'SC ŽSK - HPOV', 'SC KSK - KS', 'SC ŽSK - LIP', 'SUC PSK - SK',
       'SC KSK - SNV', 'RSUC NR - LV', 'SUC PSK - SL', 'RSUC NR - NR',
       'SC ŽSK - KYS', 'SC KSK - MI', 'SC BSK', 'SC ŽSK - TUR',
       'SUC TTSK - SE', 'NDS - SSUR ZV', 'SC KSK - TV', 'RSUC NR - NZ', 'VPO',
       'NDS - SSUD PB', 'SUC PSK - VT', 'SC KSK - RV', 'SUC TTSK - DS',
       'NDS - SSUD LM', 'RSUC NR - TO', 'RSUC NR - KN', 'NDS - SSUR KE',
       'NDS - SSUD TN', 'NDS - SSUD BE', 'MAG-BA', 'NDS - SSUR CA',
       'NDS - SSUD BA', 'NDS - SSUR NB', 'NDS - SSUD PO', 'NDS - SSUR GA',
       'NDS - SSUD MENG', 'NDS - SSUD MA', 'NDS - SSUD TT', 'NDS - SSUD MT',
       'MAG-KE', 'TT', 'NEZN', 'PP-HO', 'CU', 'ZA-Invest', 'BR-

In [5]:
# Filter bridges in the Nitra region with Trieda_PK as "cesta I. triedy" or "cesta II. triedy"
filtered_bridges = df_bridges_all[

    (df_bridges_all["Správca_úseku"] == 'SUC TTSK - TT') |
    (df_bridges_all["Správca_úseku"] == 'SUC TTSK - SE') |
    (df_bridges_all["Správca_úseku"] == 'SUC TTSK - DS')
]

# Reindex the DataFrame
filtered_bridges.reset_index(drop=True, inplace=True)

filtered_bridges = filtered_bridges[filtered_bridges["n_2023"] != 0]

filtered_bridges = filtered_bridges[filtered_bridges["Material"]
                                    == "prefabrikovaný predpätý betón"]

filtered_bridges = filtered_bridges[
    (filtered_bridges["n_2024"] == 4) |
    (filtered_bridges["n_2024"] == 5)
]

# Replace double space with single space in each item of the "Predmet_premostenia" column
filtered_bridges["Predmet_premostenia"] = filtered_bridges["Predmet_premostenia"].str.replace(
    "  ", " ")


# Display the filtered DataFrame
filtered_bridges.shape

(63, 44)

In [6]:
import pandas as pd

path = r"C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Mosty.xlsx"

# Load the Excel file into a DataFrame
df = pd.read_excel(path)

# Display the first few rows to ensure it loaded correctly
print(df.head())

      ID                         GPS   Dlzka   PK  Intenz
0  M6425  48.586307824, 17.824516076   10.43  499   12447
1  M6684  48.628765366, 17.719734958   12.40  499    3931
2  M4553  48.588876768, 17.810173653   71.60  499   14134
3   M679  48.683035961, 17.255918415   17.10  500    6052
4  M4864  48.655586793, 17.042874752  111.56  500    3663


In [7]:
import os

# Path to the folder with GPX files
folder_path = r"C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Routes"

# Function to extract distance and time from a GPX file


def extract_distance_time(gpx_file):
    with open(gpx_file, "r", encoding="utf-8") as file:
        gpx_content = file.read()

    # Initialize totals
    total_distance = 0.0
    total_time = 0.0

    # Extract <rtept> sections
    rtept_sections = gpx_content.split("<rtept")[1:]  # Split by <rtept>

    # Process each <rtept>
    for rtept in rtept_sections:
        if "<gh:distance>" in rtept:
            start_distance = rtept.find("<gh:distance>") + len("<gh:distance>")
            end_distance = rtept.find("</gh:distance>")
            distance = float(rtept[start_distance:end_distance])
            total_distance += distance
        if "<gh:time>" in rtept:
            start_time = rtept.find("<gh:time>") + len("<gh:time>")
            end_time = rtept.find("</gh:time>")
            time = float(rtept[start_time:end_time])
            total_time += time

    # Convert to kilometers and minutes
    total_distance_km = total_distance / 1000  # meters to kilometers
    total_time_minutes = total_time / (1000 * 60)  # milliseconds to minutes
    return total_distance_km, total_time_minutes


# Iterate over each row in the DataFrame
clear_distances = []
clear_times = []
alt_distances = []
alt_times = []

for _, row in df.iterrows():
    bridge_id = row["ID"]
    clear_file = os.path.join(folder_path, f"{bridge_id}_clear.gpx")
    alt_file = os.path.join(folder_path, f"{bridge_id}_alt.gpx")

    # Check if files exist
    if os.path.exists(clear_file):
        clear_distance, clear_time = extract_distance_time(clear_file)
    else:
        clear_distance, clear_time = None, None

    if os.path.exists(alt_file):
        alt_distance, alt_time = extract_distance_time(alt_file)
    else:
        alt_distance, alt_time = None, None

    # Append results
    clear_distances.append(clear_distance)
    clear_times.append(clear_time)
    alt_distances.append(alt_distance)
    alt_times.append(alt_time)

# Add new columns to the DataFrame
df["Clear_Distance"] = clear_distances
df["Clear_Time"] = clear_times
df["Alt_Distance"] = alt_distances
df["Alt_Time"] = alt_times

In [8]:
df["Diff_Dist"] = (df["Alt_Distance"] - df["Clear_Distance"])
df["Diff_Time"] = (df["Alt_Time"] - df["Clear_Time"])
df["Diff_Time_all"] = (df["Diff_Time"] * df["Intenz"]) / 60
df["Diff_Dist_all"] = (df["Diff_Dist"] * df["Intenz"])
df["Price"] = (df["Diff_Dist"]) * 0.31 * df["Intenz"]

In [9]:
df.iloc[0]["Clear_Distance"], df.iloc[0]["Alt_Distance"]

(np.float64(5.498015), np.float64(25.181497999999998))

In [10]:
df.iloc[0]["Clear_Time"], df.iloc[0]["Alt_Time"]

(np.float64(8.413483333333334), np.float64(22.787416666666665))

In [11]:
df["Price"].describe()

count        48.000000
mean       9918.314782
std       19659.766699
min         231.480899
25%        1193.170666
50%        3030.246840
75%       10764.790983
max      109782.402806
Name: Price, dtype: float64

In [19]:
import matplotlib.pyplot as plt

# Define the bridge ID to find
bridge_to_find = df.iloc[43]["ID"]

# Locate the bridge in the DataFrame
bridge_row = filtered_bridges[filtered_bridges["ID_mosta"] == bridge_to_find]
df_row = df[df["ID"] == bridge_to_find]

print(bridge_row.iloc[0][["ID_mosta", "Číslo_PK", "správcovské_číslo",
      "Rok_postavenia", "Predmet_premostenia", "Dĺžka_premostenia", "Okres", "n_2024"]])
print(bridge_row.iloc[0]["Názov_mosta"])
print("")
print(df_row.iloc[0])

ID_mosta                         M4217
Číslo_PK                          1421
správcovské_číslo                    7
Rok_postavenia                    1985
Predmet_premostenia              Kanál
Dĺžka_premostenia                 22.0
Okres                  Dunajská Streda
n_2024                               4
Name: 57, dtype: object
1421-007 most cez priesakový kanál VD  v k.ú. Vojka nad Dunajom v km 16,676

ID                                     M4217
GPS               47.960365525, 17.412817251
Dlzka                                   22.0
PK                                      1421
Intenz                                  2251
Clear_Distance                     19.621699
Clear_Time                         17.668383
Alt_Distance                       20.521371
Alt_Time                           26.446533
Diff_Dist                           0.899672
Diff_Time                            8.77815
Diff_Time_all                     329.326928
Diff_Dist_all                    2025.161672
Pr

In [13]:
bridge_to_find

'M1075'

In [21]:
import xml.etree.ElementTree as ET
import folium

import os

folder_path = r"C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Routes"

# List all files in the folder
files = os.listdir(folder_path)

# Define the bridge ID
bridge_id = bridge_to_find

# Filter the files for the specific bridge
clear_route = None
alternative_route = None

for file in files:
    if file.startswith(bridge_id):
        if "_clear" in file:
            clear_route = os.path.join(folder_path, file)
        elif "_alt" in file:
            alternative_route = os.path.join(folder_path, file)

# Assign variables for clear and alternative routes
print(f"Clear route for {bridge_id}: {clear_route}")
print(f"Alternative route for {bridge_id}: {alternative_route}")


def extract_coordinates(gpx_file):
    tree = ET.parse(gpx_file)
    root = tree.getroot()
    namespace = {"default": "http://www.topografix.com/GPX/1/1"}
    coordinates = []
    for trkpt in root.findall(".//default:trkpt", namespace):
        lat = float(trkpt.attrib['lat'])
        lon = float(trkpt.attrib['lon'])
        coordinates.append((lat, lon))
    return coordinates


# Get bridge latitude and longitude
bridge_lat = bridge_row.iloc[0]["Zemepisná_šírka"]
bridge_lon = bridge_row.iloc[0]["Zemepisná_dĺžka"]
bridge_name = bridge_row.iloc[0]["Názov_mosta"]

# Extract coordinates for clear and alternative routes
clear_coordinates = extract_coordinates(clear_route)
alt_coordinates = extract_coordinates(alternative_route)

# Create a Folium map centered on the first point of the clear route
route_map = folium.Map(location=clear_coordinates[0], zoom_start=14)

# Add a marker for the bridge
folium.Marker(
    location=[bridge_lat, bridge_lon],
    tooltip="Bridge Location",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(route_map)

# Add the clear route to the map
folium.PolyLine(clear_coordinates, color="green", weight=5,
                opacity=0.8, tooltip="Clear Route").add_to(route_map)

# Add the alternative route to the map
folium.PolyLine(alt_coordinates, color="red", weight=5,
                opacity=0.8, tooltip="Alternative Route").add_to(route_map)

route_map.save(f"most.html")

Clear route for M4217: C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Routes\M4217_clear.gpx
Alternative route for M4217: C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Routes\M4217_alt.gpx


In [15]:
print(f"Clear route for {bridge_id}: {clear_route}")
print(f"Alternative route for {bridge_id}: {alternative_route}")


# Extract data for both routes
clear_distances, clear_elevations = extract_elevation_data(clear_route)
alt_distances, alt_elevations = extract_elevation_data(alternative_route)

# Calculate the elevation range
min_elevation = min(min(clear_elevations), min(alt_elevations))
max_elevation = max(max(clear_elevations), max(alt_elevations))

# Add a buffer of ±10 meters
y_min = min_elevation - 5
y_max = max_elevation + 5

# Create a Plotly figure
fig = go.Figure()

# Add the clear route's elevation profile
fig.add_trace(go.Scatter(
    x=clear_distances,
    y=clear_elevations,
    mode="lines",
    name="Priama trasa",
    line=dict(color="green"),
    fill='tozeroy',  # Fill area under the line to the x-axis (y=0)
    fillcolor='rgba(8, 132, 9, 0.3)'  # Transparent blue fill
))

# Add the alternative route's elevation profile
fig.add_trace(go.Scatter(
    x=alt_distances,
    y=alt_elevations,
    mode="lines",
    name="Obchádzka",
    line=dict(color="red"),
    fill='tozeroy',  # Fill area under the line to the x-axis (y=0)
    fillcolor='rgba(255, 99, 71, 0.3)'  # Transparent red fill
))


# Update the layout
fig.update_layout(
    title=f"Výškové profily priamej trasy a obchádzky mosta {bridge_to_find}",
    xaxis_title="Vzdialenosť (km)",
    yaxis_title="Nadmorská výška (m)",
    yaxis=dict(range=[y_min, y_max]),  # Set y-axis limits
    legend=dict(
        title="Typ",
        orientation="h",  # Horizontal legend
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.25
    ),
    template="plotly_white"
)

# Show the plot
fig.show()

Clear route for M1075: C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Routes\M1075_clear.gpx
Alternative route for M1075: C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Routes\M1075_alt.gpx


In [16]:
df

,ID,GPS,Dlzka,PK,Intenz,Clear_Distance,Clear_Time,Alt_Distance,Alt_Time,Diff_Dist,Diff_Time,Diff_Time_all,Diff_Dist_all,Price
0,M6425,"48.586307824, 17.824516076",10.43,499,12447,5.498015,8.413483,25.181498,22.787417,19.683483,14.373933,2981.872470,245000.312901,75950.096999
1,M6684,"48.628765366, 17.719734958",12.40,499,3931,19.082168,26.423567,41.028272,47.413117,21.946104,20.989550,1375.165351,86270.134824,26743.741795
2,M4553,"48.588876768, 17.810173653",71.60,499,14134,8.363006,11.456083,33.418672,43.640850,25.055666,32.184767,7581.658201,354136.783244,109782.402806
3,M679,"48.683035961, 17.255918415",17.10,500,6052,18.211967,18.445983,42.261829,36.516217,24.049862,18.070233,1822.684202,145549.764824,45120.427095
4,M4864,"48.655586793, 17.042874752",111.56,500,3663,14.761270,17.814867,24.561341,27.218483,9.800071,9.403617,574.090797,35897.660073,11128.274623
5,M3839,"48.679414265, 17.346768217",15.00,500,7810,3.450708,6.405067,4.141911,9.731933,0.691203,3.326867,433.047144,5398.295430,1673.471583
6,M5081,"48.766953845, 17.422432832",17.50,500,882,10.169312,10.334317,13.036064,12.876333,2.866752,2.542017,37.367645,2528.475264,783.827332
7,M5770,"48.682916282, 17.272772338",12.50,500,6843,8.791168,10.359217,13.936662,15.169183,5.145494,4.809967,548.576698,35210.615442,10915.290787
8,M5237,"48.764369927, 17.408896343",16.65,500,882,7.334021,7.879033,10.200690,10.421000,2.866669,2.541967,37.366910,2528.402058,783.804638
9,M299,"48.629589908, 17.484713495",12.50,501,1310,13.123044,14.487833,21.743988,23.907900,8.620944,9.420067,205.671456,11293.436640,3500.965358


In [17]:
filtered_bridges[["ID_mosta", "Číslo_PK", "Rok_postavenia",
                  "Zemepisná_šírka", "Zemepisná_dĺžka", "Dĺžka_premostenia"]].head()

,ID_mosta,Číslo_PK,Rok_postavenia,Zemepisná_šírka,Zemepisná_dĺžka,Dĺžka_premostenia
0,M6252,507,1969,48.186735,17.712248,12.00
10,M6684,499,1973,48.628765,17.719735,12.40
19,M4553,499,1980,48.588877,17.810174,71.60
20,M6425,499,1980,48.586308,17.824516,10.43
23,M4864,500,1989,48.655587,17.042875,111.56


In [18]:
filtered_bridges["Číslo_PK"] = filtered_bridges["Číslo_PK"].astype(int)
filtered_bridges = filtered_bridges.sort_values(by="Číslo_PK", ascending=True)
filtered_bridges = filtered_bridges.reset_index(drop=True)

for _, item in filtered_bridges.iterrows():
    print(f'{item["ID_mosta"]};{item["Zemepisná_šírka"]}, {
          item["Zemepisná_dĺžka"]};{item["Dĺžka_premostenia"]};{item["Číslo_PK"]}')

M6684;48.628765366, 17.719734958;12.4;499
M4553;48.588876768, 17.810173653;71.6;499
M6425;48.586307824, 17.824516076;10.43;499
M4864;48.655586793, 17.042874752;111.56;500
M5770;48.682916282, 17.272772338;12.5;500
M679;48.683035961, 17.255918415;17.1;500
M3839;48.679414265, 17.346768217;15.0;500
M5237;48.764369927, 17.408896343;16.65;500
M5081;48.766953845, 17.422432832;17.5;500
M299;48.629589908, 17.484713495;12.5;501
M6252;48.186735332, 17.712247652;12.0;507
M283;47.975763942, 17.609488359;28.2;507
M6245;47.988603318, 17.614554817;159.26;507
M6224;48.030810689, 17.692676778;10.0;507
M7235;48.033939412, 17.695958038;17.2;507
M2332;48.25945, 17.74539;19.34;507
M2276;48.128819014, 17.478579685;91.14;510
M5118;48.438117979, 17.740006113;82.82;513
M7011;48.434617991, 17.757686108;78.84;513
M7554;48.495879831, 17.605489791;11.7;560
M141;47.994551211, 17.798279831;50.5;561
M818;47.979157779, 17.786647552;19.6;561
M4428;47.906083454, 17.811478314;22.0;561
M5666;48.112737762, 17.319021838;10.6